In [17]:

from splinter import Browser
from bs4 import BeautifulSoup
import time
import pymongo
import requests
# from splinter.exceptions import ElementDoesNotExist
import pandas as pd


In [18]:


def init_browser():

        executable_path = {"executable_path": "chromedriver.exe"}
        return Browser("chrome", **executable_path, headless=False)




In [19]:
address = 'Holston'
browser = init_browser()
url = 'https://property.spatialest.com/nc/durham/#/'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

results = soup.find_all('input', class_='search')
print(results)

[<input class="search" id="primary_search" placeholder="Enter search term..." type="text" value=""/>]


In [20]:
browser.find_by_id('primary_search').first.fill(address)

In [22]:
browser.click_link_by_partial_href('#') 

In [23]:
data = browser.url
browser.visit(data)

property_link1 = browser.html
prop_link1 = BeautifulSoup(property_link1, 'html.parser')
prop_link1
prop_list = []
addresses = prop_link1.findAll('div', class_="item-1")
for address in addresses:
    prop_list.append(address.text)
prop_list

['4201 HOLSTON DR',
 '4203 HOLSTON DR',
 '4205 HOLSTON DR',
 '4207 HOLSTON DR',
 '4209 HOLSTON DR',
 '4211 HOLSTON DR',
 '4213 HOLSTON DR',
 '4215 HOLSTON DR',
 '4217 HOLSTON DR',
 '4219 HOLSTON DR',
 '4221 HOLSTON DR',
 '4223 HOLSTON DR',
 '1218 MARNE AVE',
 '232 MORNINGSIDE DR',
 '112 FILIGREE WAY']

In [24]:
next_list =  browser.find_by_text('Next')


In [25]:
if len(next_list)>0:
    browser.click_link_by_partial_text('Next') 
    time.sleep(3)
    data1 = browser.url
    browser.visit(data1)

    property_link1 = browser.html
    prop_link1 = BeautifulSoup(property_link1, 'html.parser')
    prop_link1

    addresses1 = prop_link1.findAll('div', class_="item-1")
    for address1 in addresses1:
        prop_list.append(address1.text)
    prop_list
else:
    print("done")

done


In [8]:
prop_list1 = [x for x in prop_list if not x.startswith('13')]
prop_list1

['1242 MARNE AVE',
 '1240 MARNE AVE',
 '1236 MARNE AVE',
 '1232 MARNE AVE',
 '1228 MARNE AVE',
 '1224 MARNE AVE',
 '1220 MARNE AVE',
 '1214 MARNE AVE',
 '1210 MARNE AVE',
 '1206 MARNE AVE',
 '1202 MARNE AVE',
 '1201 MARNE AVE',
 '1205 MARNE AVE',
 '1209 MARNE AVE',
 '1213 MARNE AVE',
 '1217 MARNE AVE',
 '1221 MARNE AVE',
 '1225 MARNE AVE',
 '1229 MARNE AVE',
 '1233 MARNE AVE',
 '1237 MARNE AVE',
 '1218 MARNE AVE']

In [9]:
browser.click_link_by_partial_href('spatialest')
time.sleep(3)
prop_mailing1 = []    
address = []
price = []
date = []
while len(prop_list1)>0:
    try:
        for prop in prop_list1:
            browser.find_by_id('primary_search').first.fill(prop)
            time.sleep(3)
            browser.click_link_by_partial_href('#') 
            time.sleep(5)
            data = browser.url
            browser.visit(data)
            time.sleep(3)
            property_link3 = browser.html
            prop_link = BeautifulSoup(property_link3, 'html.parser')
            prop_mailing = prop_link.find('div', class_='mailing')
            prop_mailing1.append(prop)
            address.append(prop)
            prop_table = prop_link.findAll('table')[3]
            price_table = prop_table.findAll('td')[1]
            date_table = prop_table.findAll('td')[0]
            for x in price_table:
                price.append(x)
            for y in date_table:
                date.append(y)
            browser.click_link_by_partial_href('spatialest')
            time.sleep(3)
            prop_list1.remove(prop)
            print(len(prop_list1))
    except:
        
        prop_list1.remove(prop)
        print(len(prop_list1))
        browser.click_link_by_partial_href('spatialest')
        time.sleep(2)

21
20
19
18
17
16
15
14
13
12
11
10
9
8
7
6
5
4
3
2
1
0


In [10]:
df = pd.DataFrame(columns=['Address', 'Price', 'Date'])
df['Address']=address
df['Price']=price
df['Date']=date
df

,Address,Price,Date
0,1242 MARNE AVE,"162,500",2016 Q1
1,1236 MARNE AVE,"72,000",03/22/2010
2,1228 MARNE AVE,"57,000",02/28/2011
3,1220 MARNE AVE,"90,500",03/26/2015
4,1210 MARNE AVE,"72,500",11/29/2011
5,1202 MARNE AVE,"162,500",2016 Q1
6,1205 MARNE AVE,"162,500",2016 Q1
7,1213 MARNE AVE,"102,000",02/26/2016
8,1221 MARNE AVE,"70,000",03/12/2010
9,1229 MARNE AVE,"92,500",05/20/2015


In [11]:
df.to_csv('Barringer3.csv', encoding='utf-8', index=False)

In [27]:
df1 = pd.read_csv('Barringer.csv')

In [28]:
df1

,Address,Price,Last Sales Date
0,1222 RED ASH CIR,"$102,000.00",11/13/2013
1,1220 RED ASH CIR,"$127,500.00",5/15/2002
2,1218 RED ASH CIR,"$120,000.00",2/19/2002
3,1216 RED ASH CIR,"$127,500.00",2/19/2002
4,1214 RED ASH CIR,"$130,000.00",12/18/2001
5,1212 RED ASH CIR,"$130,000.00",5/27/2008
6,1210 RED ASH CIR,"$124,500.00",9/3/2003
7,1208 RED ASH CIR,"$115,000.00",5/2/2002
8,1207 RED ASH CIR,"$120,000.00",3/31/2017
9,1204 RED ASH CIR,"$128,000.00",5/21/2002


In [29]:
df1.dropna()

,Address,Price,Last Sales Date
0,1222 RED ASH CIR,"$102,000.00",11/13/2013
1,1220 RED ASH CIR,"$127,500.00",5/15/2002
2,1218 RED ASH CIR,"$120,000.00",2/19/2002
3,1216 RED ASH CIR,"$127,500.00",2/19/2002
4,1214 RED ASH CIR,"$130,000.00",12/18/2001
5,1212 RED ASH CIR,"$130,000.00",5/27/2008
6,1210 RED ASH CIR,"$124,500.00",9/3/2003
7,1208 RED ASH CIR,"$115,000.00",5/2/2002
8,1207 RED ASH CIR,"$120,000.00",3/31/2017
9,1204 RED ASH CIR,"$128,000.00",5/21/2002


In [30]:
df1 = df1.loc[df1['Date'] !='2016 Q1']

KeyError: 'Date'

In [26]:
df1

,Address,Price,Date
1,1236 MARNE AVE,"72,000",03/22/2010
2,1228 MARNE AVE,"57,000",02/28/2011
3,1220 MARNE AVE,"90,500",03/26/2015
4,1210 MARNE AVE,"72,500",11/29/2011
7,1213 MARNE AVE,"102,000",02/26/2016
8,1221 MARNE AVE,"70,000",03/12/2010
9,1229 MARNE AVE,"92,500",05/20/2015
10,1237 MARNE AVE,"76,000",11/13/2015
11,1240 MARNE AVE,"78,600",06/27/2012
12,1224 MARNE AVE,"139,628",02/28/2011
